# IMPORT ALL LIBARARIES

In [1]:
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
from PIL import Image
import warnings
warnings.warn("Model.fit_generator")
warnings.warn("message")


<ipython-input-1-e79b4946a86f>:15: UserWarning: Model.fit_generator
  warnings.warn("Model.fit_generator")
<ipython-input-1-e79b4946a86f>:16: UserWarning: message
  warnings.warn("message")


In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [3]:
TRAINING_DIR = r"C:\Users\SOMAY\Downloads\face-mask-dataset\train\train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR =r"C:\Users\SOMAY\Downloads\face-mask-dataset\test\test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [4]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [5]:
history = model.fit(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
132/132 [==============================] - 44s 332ms/step - loss: 0.6111 - acc: 0.7072 - val_loss: 0.2094 - val_acc: 0.9227
INFO:tensorflow:Assets written to: model2-001.model\assets
Epoch 2/10
132/132 [==============================] - 44s 331ms/step - loss: 0.3240 - acc: 0.8707 - val_loss: 0.1406 - val_acc: 0.9639
INFO:tensorflow:Assets written to: model2-002.model\assets
Epoch 3/10
132/132 [==============================] - 43s 323ms/step - loss: 0.2760 - acc: 0.8981 - val_loss: 0.1989 - val_acc: 0.9124
Epoch 4/10
132/132 [==============================] - 42s 319ms/step - loss: 0.2808 - acc: 0.8951 - val_loss: 0.0887 - val_acc: 0.9691
INFO:tensorflow:Assets written to: model2-004.model\assets
Epoch 5/10
132/132 [==============================] - 43s 322ms/step - loss: 0.2599 - acc: 0.8935 - val_loss: 0.1215 - val_acc: 0.9485
Epoch 6/10
132/132 [==============================] - 43s 326ms/step - loss: 0.2114 - acc: 0.9217 - val_loss: 0.0812 - val_acc: 0.9639
INFO:tensorfl

In [6]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("model2-009.model")

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier(r'C:\Users\SOMAY\Downloads\cascade1\cascade\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()